# [WIP] SCIS 2026 "セキュリティ用オープンLLMエージェントシステムのエンジニアリングガイド" のケーススタディ実装

本ノートブックは，上記論文の第4章「ケーススタディ」における実装を示すものです．ただし，論文中の記述に対して本ノートブック内の実装には以下の相違点があります．なお，これらはLLMエージェントの論理的な設計に影響を及ぼすものではありません．

- Google Colaboratoryが使用するTesla T4 GPUは世代が古くSGLangが実質的に動作しないため，推論フレームワークにvLLMを使用しています．
- Google Colaboratoryが使用するTesla T4 GPUは世代が古く小数精度としてFP8フォーマットをサポートしないため，KVキャッシュ量子化は使用していません．



## 初期セットアップ（必ず実行してください）

まず，GPUが使用可能なランタイムであることを確認するために以下のセルを実行してください．

In [ ]:
from IPython.core.display import HTML, display

html_alertbox = HTML("""
<div class="alert">
  <p>このランタイムではGPUが使用できません．画面の上にある「ランタイム」→「ランタイムのタイプを変更」→「T4 GPU」を選択して，GPUランタイムに切り替えてください．</p>
</div>

<style>
.alert {
  padding: 20px;
  background-color: #f44336;
  color: white;
  margin-bottom: 15px;
}
</style>
""")

import torch
if not torch.cuda.is_available():
  display(html_alertbox)
  raise Exception("GPUが使用できません．GPUランタイムに切り替えてください．")

次に，セルの出力が長い場合に折り返されるように設定します．

In [ ]:
# Enable wrapping for the output cells
from IPython.display import HTML, display
from IPython.core.getipython import get_ipython
def set_css(_):
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap !important;
        word-wrap: break-word !important;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

最後に，以下のセルで必要なライブラリをインストールします．

In [ ]:
!git clone https://github.com/t0d4/SecurityLLMAgentDesignGuide-SCIS2026.git
%cd SecurityLLMAgentDesignGuide-SCIS2026
!uv pip install --system -r requirements.txt

!uv pip install --system vllm==0.11.0 langchain langchain-deepseek pydantic

In [ ]:
%%bash

OPENAI_BASE_URL="http://localhost:11434/v1" \
    OPENAI_API_KEY="sk-dummy" \
    python -m openai api chat.completions.create \
    -g user "GPUに関する俳句を詠んでください．その後に短い解説をつけてください．" \
    -m "qwen3:8b"

## LLM推論フレームワークの起動

GPTQによりInt4量子化された `Qwen3-4B-Thinking-2507` を動作させるサーバを起動します．初回実行時にはモデルをダウンロードする必要があるため時間がかかります．

In [ ]:
%%bash

# SGLangは既にGoogle ColabのTesla T4 GPUをサポート対象外としており低速にしか動作しないため，ここでは使用しません．
# nohup python -m sglang.launch_server \
#   --model-path JunHowie/Qwen3-4B-Thinking-2507-GPTQ-Int4 \
#   --mem-fraction-static 0.8 \
#   --context-length 32768 \
#   --reasoning-parser qwen3 \
#   --tool-call-parser qwen \
#   --kv-cache-dtype fp8_e4m3 &

# Google Colab上では代わりにvLLMを使用します
# For details of reasoning parser configuration, please visit https://github.com/vllm-project/vllm/issues/26239#issuecomment-3385327094
nohup uv run vllm serve JunHowie/Qwen3-4B-Thinking-2507-GPTQ-Int4 \
  --served-model-name Qwen3-4B-Thinking-2507 \
  --gpu-memory-utilization 0.85 \
  --max-model-len 32768 \
  --reasoning-parser deepseek_r1 \
  --enable-auto-tool-choice \
  --tool-call-parser hermes \
  --host 127.0.0.1 --port 8000 > nohup.out 2>&1 &

# vLLMの起動が完了するまで待機します (ログ出力はターミナルから `tail -f nohup.out` で確認できます)
until curl -sf http://127.0.0.1:8000/health; do
  sleep 5
done
echo "vLLM startup completed."

## Supervisor-Worker型マルチエージェントシステムの実装

まず，vLLMのサーバをバックエンドとするクライアントを定義します．

注) 以下では `ChatDeepSeek` を使用していますが，バックエンドのモデルはDeepSeekではなく上で述べた通りQwen3-4B-Thinking-2507です．これは以下の理由によります:

- vLLMはOpenAI互換APIを提供するため `ChatOpenAI` からも接続可能だが，その場合[応答にReasoning途中の出力トークン (Reasoning Tokens) を含めないOpenAI APIの仕様](https://platform.openai.com/docs/guides/reasoning#reasoning-summaries)によりReasoning Tokensが取得できない
- DeepSeek APIはOpenAI互換APIとして提供されており，かつReasoning Tokensをユーザへの応答に含める
- したがって， `ChatDeepSeek` を使うことでvLLMに接続でき，かつReasoning Tokensを取得できる

In [ ]:
from langchain_deepseek import ChatDeepSeek

MODEL_NAME = "Qwen3-4B-Thinking-2507"

llm = ChatDeepSeek(
    model=MODEL_NAME,
    name=MODEL_NAME,
    api_base="http://127.0.0.1:8000/v1",
    api_key="sk-dummy",
    temperature=0.6,
    top_p=0.95,
    extra_body={
        "top_k": 20,
        "separate_reasoning": True
    },
)

### Deobfuscatorの実装

Deobfuscatorに与えるツールを実装します．ここでは，base64ペイロードをデコードするためのツールとして `decode_base64_payload` を用意します．

In [ ]:
import base64

from pydantic import BaseModel, Field
from langchain_core.tools import tool

class DecodeBase64PayloadInput(BaseModel):
    payload: str = Field(description="A payload to decode")


@tool(args_schema=DecodeBase64PayloadInput)
def decode_base64_payload(payload: str) -> str:
    """Decode base64-encoded strings.
    Runtime decoding of base64 data is done using the `base64.b64decode()` function.
    If you encounter base64-encoded data, this tool will decode it for you.
    When decoding fails, the error message will be shown to you.

    Args:
        payload (str): a string encoded in base64 format

    Returns:
        str: Decoded content or error message with forensic details

    Example success:
        "Hello, World!" (when decoding "SGVsbG8sIFdvcmxkIQ==")
    """
    try:
        import base64

        decoded = base64.b64decode(payload.encode()).decode().replace("\0", "")
    except Exception as e:
        return (
            f"Failed to decode the base64 string. Error message is the following: {e}. "
            "This could be due to incorrect payload, but also consider the possibility that this could be a random string and may not be a base64 string."
        )
    else:
        return decoded

Deobfuscator本体をエージェントとして定義します．

In [ ]:
from langchain.agents import create_agent


# Workerエージェントが最終出力を行う際に従うべきフォーマットを指定します．
# https://docs.langchain.com/oss/python/langchain/structured-output
class WorkerResponse(BaseModel):
    """Standard format for a worker agent response"""
    detailed_report: str = Field(description="detailed, markdown-style report containing all details discovered during the task")
    short_summary: str = Field(description="minimal-length focused summary articulating the essential findings")


deobfuscator = create_agent(
    model=llm,
    tools=[decode_base64_payload],
    system_prompt="",
    response_format=WorkerResponse,
)